#NMT Attack

Welcome to the NMT-Attack Notebook. Here we have detailed the process of reproducing our results for our paper : Lost in Translation

We have 3 sections to this notebook. 

The first involves installation of dependencies.

The second involves running the attack.

The third involves analysis of our results.

**Please upload the TextAttack folder to the files section of the notebook, situated on the left of the code section**


##We also provide the provision to reproduce all 3 attacks we have used. 

For TextFooler, PWWS and TextBugger, please edit the models and tokenisers (as detailed in the NMT Attack section) in the **greedy_word_swap_wir.py** search methods file. 

For example, to work with Rotten Tomatoes dataset, please update the model and tokeniser to 

**original_tokenizer = AutoTokenizer.from_pretrained("textattack/bert-base-uncased-rotten-tomatoes")**

**original_model = AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-rotten-tomatoes")**

**model = HuggingFaceModelWrapper(original_model,original_tokenizer)**


This must be updated in both the colab notebook as well as the greedy_word_swap_wir.py file





In [ ]:
%cd /path/to/TextAttack/ #Run this instruction in every iteration of this notebook.

##Pip Requirements(Only run initially)

You will in most cases have to restart runtime after this. Once you restart runtime, you DO NOT have to run this section again.

In [ ]:
!pip install -e .

In [ ]:
!pip install tensorflow_datasets
!pip3 install -U easynmt
# !pip install tensorflow-gpu==2.1.0


In [ ]:
!pip install captum

##NMT-Attack

In [ ]:
%%time
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import torch
import textattack
from textattack.models.wrappers import ModelWrapper
from textattack.datasets import HuggingFaceDataset
from textattack.attack_recipes import TextFoolerJin2019
from textattack.attack_recipes import BAEGarg2019
import pandas as pd

In [ ]:
from captum.attr import IntegratedGradients, LayerConductance, LayerIntegratedGradients, LayerDeepLiftShap, InternalInfluence, LayerGradientXActivation
from captum.attr import visualization as viz

In [ ]:
from textattack.datasets import HuggingFaceDataset
from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack.models.wrappers import ModelWrapper
from transformers import AutoModelForSequenceClassification, AutoTokenizer

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")

print(device)

###Important Note

The following code block specifies the particular model you would like to use for the attack, as well as the dataset to be used for the attack. 

###Make sure you specify the exact same tokenizer and model in the specific search method used by the algorithm. For convenience, we have already added a model in the search method of the algorithm for your convenience.

In [ ]:
dataset = HuggingFaceDataset("rotten_tomatoes", None, "test")
original_tokenizer = AutoTokenizer.from_pretrained("textattack/bert-base-uncased-rotten-tomatoes")
original_model = AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-rotten_tomatoes")
model = HuggingFaceModelWrapper(original_model,original_tokenizer)


Below, chose the attack you would look to use. The options are:


1.   TextFoolerJin2019
2.   TextBuggerLi2018

1.   PWWSRen2019


Attach this to the build function, and run the cell.



In [ ]:
from textattack import Attacker
from textattack.attack_recipes import TextBuggerLi2018
attack = TextFoolerJin2019.build(model)
attack_args = textattack.AttackArgs(num_examples=1000)
attacker = Attacker(attack, dataset,attack_args)
x=attacker.attack_dataset()

Please store the last robust example array in the output, containing the number of robust examples for each examples, in the array below

In [ ]:
robust_ex=[1, 19, 1, 2, 1, 0, 18, 23, 0, 8, 1, 5, 3, 16] #Size of this array= number of attacks successful.

In [ ]:
countt=0
for i in range(len(textfooler_yelp)):
  if textfooler_yelp[i]!=0:
    # print(org_list[i])
    # print(i)
    countt+=1;
print(countt)

#Analysis section

In [ ]:
list_german=[]
list_french=[]
list_spanish=[]
org_list=[]
unadv=[]

In [ ]:
for i in x:
  if(type(i)==textattack.attack_results.skipped_attack_result.SkippedAttackResult):
    continue;
  if(type(i)==textattack.attack_results.failed_attack_result.FailedAttackResult):
    continue;
  org_list.append(i.perturbed_text())
  unadv.append(i.original_text())

In [ ]:
df_d=pd.DataFrame({"original":unadv,
                 "adversarial":org_list,
})

In [ ]:
df_d.to_csv('/content/drive/MyDrive/labels/temp_results.csv') #store temporarily since the colab notebook might disconnect depending on GPU allocation provided

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/labels/temp_results.csv')
org_list= df['adversarial']
unadv= df['original']

In [ ]:
c=[]
from nltk.tokenize import word_tokenize #to match tokenisation done during inference
for t in org_list:
  tokens = word_tokenize(t)
  # remove all tokens that are not alphabetic
  words = [word for word in tokens if word.isalpha()]
  z=' '.join(words)
  c.append(z)

In [ ]:
org_list=c

In [ ]:
trans_german=[]
trans_french=[]
trans_spanish=[]

In [ ]:
from easynmt import EasyNMT
model_lang = EasyNMT('opus-mt')

In [ ]:
list_german=model_lang.translate(org_list,source_lang='en', target_lang='de')
list_french=model_lang.translate(org_list,source_lang='en', target_lang='fr')
list_spanish=model_lang.translate(org_list,source_lang='en', target_lang='es')

In [ ]:
trans_german=model_lang.translate(list_german,source_lang='de', target_lang='en')
trans_french=model_lang.translate(list_french,source_lang='fr', target_lang='en')
trans_spanish=model_lang.translate(list_spanish,source_lang='es',target_lang='en')

In [ ]:
span=[]
for i in range(len(trans_spanish)):
  span.append(model([trans_spanish[i]]))
st=[]
for i in range(len(trans_spanish)):
  st.append(model([trans_german[i]]))
frenc=[]
for i in range(len(trans_spanish)):
  frenc.append(model([trans_french[i]]))

In [ ]:

count=0
for i in range(len(st)):
  if (torch.argmax(st[i])==0):
    count=count+1

count_f=0
for i in range(len(frenc)):
  if (torch.argmax(frenc[i])==0):
    count_f=count_f+1

count_s=0
for i in range(len(span)):
  if (torch.argmax(span[i])==0):
    count_s=count_s+1
print(count_s)
  

In [ ]:
count=0
for i in range(len(tf_50_yelp)):
  if tf_50_yelp[i]!=0:
    count+=1;
print(count)

In [ ]:
score_german=[]
score_french=[]
score_spanish=[]
for i in range(len(st)):
  if (torch.argmax(st[i])==1):
    score_german.append(True)
  else:
    score_german.append(False)
for i in range(len(frenc)):
  if (torch.argmax(frenc[i])==1):
    score_french.append(True)
  else:
    score_french.append(False)
for i in range(len(span)):
  if (torch.argmax(span[i])==1):
    score_spanish.append(True)
  else:
    score_spanish.append(False)

THe block below provides how many examples are robust to NMT for a particular language (selected from score_(language)

In [ ]:
countt=0
for i in range(len(score_spanish)):
  if score_spanish[i]==True and robust_ex==0:
    # print(org_list[i])
    # print(i)
    countt+=1;
print(countt)

In [ ]:
df=pd.DataFrame({"original":unadv,
                 "adversarial":org_list,
                 "back_adv_german":trans_german,
                 "back_adv_french":trans_french,
                 "back_adv_spanish":trans_spanish,
                 "german_change":score_german,
                 "french_change":score_french,
                 "spanish_change":score_spanish})

In [ ]:
df.to_csv('/content/drive/MyDrive/labels/file_success.csv')

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/TextAttackNB/file_success.csv')

In [ ]:
df_vis=df

##Quantitative Results

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/TextAttackNB/file_success.csv')

# df_int=pd.read_csv('/content/drive/MyDrive/TextAttackNB/40cand_interpret.csv')

In [ ]:
import torch

In [ ]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)


In [ ]:
sim_metric=torch.nn.CosineSimilarity(dim=-1)

In [ ]:
sim
for i in range(len(df)):
  x=df['original'][0]
  y=df['adversarial'][0]

In [ ]:
x

In [ ]:
y

In [ ]:
z=np.array(model([x]))
a=np.array(model([y]))

In [ ]:
sent1=torch.tensor(z[0])
sent2=torch.tensor(a[0])

In [ ]:
z=sim_metric(sent1,sent2).detach().cpu().numpy()

In [ ]:
z

In [ ]:
print(tf.keras.losses.cosine_similarity(
    d,
      f
))

In [ ]:
d=model([x])
f=model([y])

In [ ]:
np.sum(t)/110

In [ ]:
# USE Scores for Delete
t=[]
for i in range(len(df)):
  if new_del[i]==0:
    continue;
  x=df['original'][i]
  y=df['adversarial'][i]
  sent1=torch.tensor(np.array(model([x]))[0])
  sent2=torch.tensor(np.array(model([y]))[0])
  sim=sim_metric(sent1,sent2)
  t.append(sim.detach().cpu().numpy())

In [ ]:
int_t=[]
for i in range(len(df)):
  x=df_int['original'][i]
  y=df_int['adversarial'][i]
  sent1=torch.tensor(np.array(model([x]))[0])
  sent2=torch.tensor(np.array(model([y]))[0])
  sim=sim_metric(sent1,sent2)
  int_t.append(sim.detach().cpu().numpy())

In [ ]:
count=0;
bf=0
diff_list=[]
for i in range(len(int_t)):
  diff_list.append(int_t[i]-t[i])
  if(diff_list[i]<=0):
    bf+=diff_list[i]
    count+=1;

In [ ]:
bf/count

In [ ]:
import numpy as np
def levenshtein_ratio_and_distance(s, t, ratio_calc = False):
    """ levenshtein_ratio_and_distance:
        Calculates levenshtein distance between two strings.
        If ratio_calc = True, the function computes the
        levenshtein distance ratio of similarity between two strings
        For all i and j, distance[i,j] will contain the Levenshtein
        distance between the first i characters of s and the
        first j characters of t
    """
    # Initialize matrix of zeros
    rows = len(s)+1
    cols = len(t)+1
    distance = np.zeros((rows,cols),dtype = int)

    # Populate matrix of zeros with the indeces of each character of both strings
    for i in range(1, rows):
        for k in range(1,cols):
            distance[i][0] = i
            distance[0][k] = k

    # Iterate over the matrix to compute the cost of deletions,insertions and/or substitutions    
    for col in range(1, cols):
        for row in range(1, rows):
            if s[row-1] == t[col-1]:
                cost = 0 # If the characters are the same in the two strings in a given position [i,j] then the cost is 0
            else:
                # In order to align the results with those of the Python Levenshtein package, if we choose to calculate the ratio
                # the cost of a substitution is 2. If we calculate just distance, then the cost of a substitution is 1.
                if ratio_calc == True:
                    cost = 2
                else:
                    cost = 1
            distance[row][col] = min(distance[row-1][col] + 1,      # Cost of deletions
                                 distance[row][col-1] + 1,          # Cost of insertions
                                 distance[row-1][col-1] + cost)     # Cost of substitutions
    if ratio_calc == True:
        # Computation of the Levenshtein Distance Ratio
        Ratio = ((len(s)+len(t)) - distance[row][col]) / (len(s)+len(t))
        return Ratio
    else:
        # print(distance) # Uncomment if you want to see the matrix showing how the algorithm computes the cost of deletions,
        # insertions and/or substitutions
        # This is the minimum number of edits needed to convert string a to string b
        return distance[row][col]

In [ ]:
levenshtein_ratio_and_distance(x,y)

In [ ]:
lev_dist=[]
for i in range(len(df)):
  if new_del[i]==0:
    continue;
  x=df['original'][i]
  y=df['adversarial'][i]
  # d=df_int['original'][i]
  # f=df_int['adversarial'][i]
  lev_dist.append(levenshtein_ratio_and_distance(x,y))

In [ ]:
np.sum(lev_dist)/110

In [ ]:
p=0
sum=0;
for i in range(len(lev_dist)):
  if lev_dist[i]<0:
    sum+=lev_dist[i]
    p=p+1

In [ ]:
!pip install bert_score
!pip install transformers

In [ ]:
import bert_score
import transformers

In [ ]:
from bert_score import score

In [ ]:
P, R, F1 = score([x], [y], lang='en', verbose=True)



In [ ]:
P, R, F1 = score(df['original'], df['adversarial'], lang='en', verbose=True)

In [ ]:
bert_score_del=[]
for i in range(len(df)):
  if new_del[i]==0:
    continue;
  x=df['original'][i]
  y=df['adversarial'][i]
  P, R, F1 = score([x], [y], lang='en', verbose=True)
  bert_score_del.append(F1)

In [ ]:
np.sum(bert_score_del)/110

In [ ]:
def Jaccard_Similarity(doc1, doc2): 
    
    # List the unique words in a document
    words_doc1 = set(doc1.lower().split()) 
    words_doc2 = set(doc2.lower().split())
    
    # Find the intersection of words list of doc1 & doc2
    intersection = words_doc1.intersection(words_doc2)

    # Find the union of words list of doc1 & doc2
    union = words_doc1.union(words_doc2)
        
    # Calculate Jaccard similarity score 
    # using length of intersection set divided by length of union set
    return float(len(intersection)) / len(union)

In [ ]:
p=[]
for i in range(len(df)):
  if new_del[i]==0:
    continue;
  x=df['original'][i]
  y=df['adversarial'][i]
  p.append(Jaccard_Similarity(x,y))

In [ ]:
np.sum(p)/110

In [ ]:
df['adversarial'][13]

In [ ]:
df['original'][10]

In [ ]:
bert_diff_list=[]
count_bert=0;
c=0
for i in range(len(bert_score_del)):
  bert_diff_list.append(bert_score_del[i]-bert_score_int[i])
  if bert_diff_list[i]>=0:
    count_bert+=bert_diff_list[i]
    c+=1